In [12]:
from scipy.io import loadmat
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler

scaler=StandardScaler()

In [13]:
##---------------DATOS DE ENTRENAMIENTO -------------------------##
##Cargar los datos con extención .mat
x = loadmat('/DatosPrueba4.mat')

##Traernos los datos del archivo .mat que nos interesan
y = x['data_tr']

##Convertir los datos en un array de Numpy
z = np.array(y)

##Convertir el Array en un DataFrame de Pandas
df = pd.DataFrame(z)

##Contar el numero de datos nulos del dataFrame
np.count_nonzero(pd.isnull(df) == True)

df.columns = ['1','2','3','4','5','6','7','8','9','10','11','12','13','14','15','16','17','18','Label']
df.sample(8)

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,Label
908,0.006190,0.014300,0.000888,0.000807,3.521494,13.090987,0.087076,0.000191,0.015516,1602.66,258.158131,1676.0,0.0000,-3.489554,11.254435,1676.0,253.0,1623.113748,0.0
615,0.011189,0.022381,0.002011,0.002377,3.062000,9.917885,0.121186,0.000120,0.024922,1043.06,568.235117,1476.0,0.0000,-0.617410,-1.540981,1476.0,222.0,1186.438536,0.0
548,0.001429,0.002162,0.000338,0.000008,3.956466,20.162835,0.015037,0.000253,0.002583,1654.56,105.563263,1676.0,0.0000,-4.624609,19.583137,1676.0,1140.0,1657.890515,0.0
218,0.351807,1.214566,0.000803,0.000961,4.400597,21.144669,8.210795,0.000147,1.258645,781.00,625.954221,543.5,467.7603,0.610409,-1.468766,1676.0,222.0,998.930178,0.0
450,0.000188,0.000367,0.000009,0.000008,2.027748,2.385262,0.001161,0.000003,0.000411,116.72,34.295825,138.0,0.0000,-0.965295,-1.078693,138.0,62.0,121.605921,1.0
414,0.018401,0.057936,0.000148,0.000062,2.824621,6.045248,0.211577,0.000084,0.060511,224.46,2.965866,222.0,0.0000,0.360502,-1.888626,228.0,222.0,224.479398,0.0
959,0.150027,0.447656,0.002750,0.003578,4.101417,18.045140,2.920582,0.000125,0.470000,1231.84,608.881119,1676.0,0.0000,-0.737978,-1.307871,1676.0,228.0,1372.755856,0.0
732,0.012387,0.036551,0.002073,0.000023,5.522256,37.725008,0.302468,0.000203,0.038419,1291.67,567.192894,1596.0,0.0000,-1.316204,-0.253547,1596.0,228.0,1409.575145,0.0


In [14]:
YEntrenamiento = df['Label']
XEntrenamiento = scaler.fit_transform(df.drop(['Label'],axis=1))

In [15]:

from sklearn.model_selection import StratifiedKFold

In [16]:
from sklearn.ensemble import GradientBoostingClassifier

In [17]:
def experimentar_gbt(num_trees, X, Y):
    """funcion que realiza experimentos de arboles de decision
    num_trees: list[int] lista con el número de arboles usado para el RF
    X: matriz con las caractersiticas
    Y: matriz de numpy con etiquetas
    retorna: dataframe con:
        - numero de arboles usados
        - eficiencia de entrenamiento
        - desviacion de estandar eficiencia de entrenamiento
        - eficiencia de prueba
        - desviacion estandar eficiencia de prueba
    """
    folds = 4
    skf = StratifiedKFold(n_splits=folds)
    resultados = pd.DataFrame()
    idx = 0
    for trees in num_trees:
        ## para almacenar los errores intermedios
        EficienciaTrain = []
        EficienciaVal = []
        for train, test in skf.split(X, Y):
            Xtrain = X[train,:]
            Ytrain = Y[train]
            Xtest = X[test,:]
            Ytest = Y[test]
            #Haga el llamado a la función para crear y entrenar el modelo usando los datos de entrenamiento
            modelo= GradientBoostingClassifier(n_estimators= trees)
            modelo.fit(Xtrain, Ytrain)
            #predecir muestras de entrenamiento
            Ytrain_pred =  modelo.predict(Xtrain)
            #predecir muestras de pruebas
            Yest = modelo.predict(Xtest)
            #Evaluamos las predicciones del modelo con los datos de test
            EficienciaTrain.append(np.mean(Ytrain_pred.ravel() == Ytrain.ravel()))
            EficienciaVal.append(np.mean(Yest.ravel() == Ytest.ravel()))

        resultados.loc[idx,'número de arboles'] = trees
        resultados.loc[idx,'eficiencia de entrenamiento'] = np.mean(EficienciaTrain)
        resultados.loc[idx,'desviacion estandar entrenamiento'] = np.std(EficienciaTrain)
        resultados.loc[idx,'eficiencia de prueba'] = np.mean(EficienciaVal)
        resultados.loc[idx,'desviacion estandar prueba'] = np.std(EficienciaVal)
        idx= idx +1
        
    return (resultados)

In [20]:
arboles = [5,10,20,50,100, 150]
resultados_gbt = experimentar_gbt(arboles, XEntrenamiento, YEntrenamiento)
resultados_gbt

,número de arboles,eficiencia de entrenamiento,desviacion estandar entrenamiento,eficiencia de prueba,desviacion estandar prueba
0,5.0,1.0,0.0,1.0,0.0
1,10.0,1.0,0.0,1.0,0.0
2,20.0,1.0,0.0,1.0,0.0
3,50.0,1.0,0.0,1.0,0.0
4,100.0,1.0,0.0,1.0,0.0
5,150.0,1.0,0.0,1.0,0.0
